# pythonでテレワークメールを解析してExcelに記入する
## ライブラリとか
1. win32com  
1. openpyxl  
1. datetime,calendar  

In [1]:
import os
import win32com.client
import openpyxl
import datetime
import calendar

# 初期値
FILE_MEMBERS = "members.txt"
FILE_HOLIDAYS = "holidays.txt"
FOLDER_ARCHIVE = "テレワーク"

### Outlook読み取り用に諸々初期化

In [2]:
app = win32com.client.Dispatch("Outlook.Application")
root = app.Session.DefaultStore.GetRootFolder()
ns = app.GetNamespace("MAPI")
inbox = ns.GetDefaultFolder(6)
messages = inbox.Items
print('outlook initialize done. inbox:',inbox.name,'Count=',messages.Count)

outlook initialize done. inbox: 受信トレイ Count= 48


In [6]:
def findTeleworkFolder(root, teleworkfolder):
    for folder in root.Folders:
        if teleworkfolder in folder.name:
            # print("find:", folder.name)
            return folder
    return nil


# tele = findTeleworkFolder(root, "テレワーク")
# print('archive folder found:',tele.name)

### メンバリストと祝日リストをロードする

In [4]:
def load_members(file_name):
    with open(file_name, "r", newline="", encoding="utf-8-sig") as f:
        members = f.read().splitlines()
    print('members:',members)
    return members

In [5]:
def load_holidays(file_name):
    with open(file_name, encoding="utf-8-sig") as f:
        lines = f.read().splitlines()
        holidays = list()
        for line in lines:
            d = datetime.date.fromisoformat(line)
            # print(d)
            holidays.append(d)
        print('holidays:',holidays[:5])
        return holidays

### 各種関数とテストコード

In [22]:
# テレワークの開始終了を知らせるメールかどうか判定する
def isTelework(item, members, date_from=None):
    if date_from != None and date_from > item.senton.date():
        return -1
    if "テレワーク" in item.subject:
        for name in members:
            if name in item.sendername:
                if "開始" in item.subject:
                    return 0
                elif "終了" in item.subject:
                    return 1
                else:
                    indexst = item.body.find("開始")
                    indexed = item.body.find("終了")
                    if indexst < 0 and indexed >= 0:
                        return 1
                    if indexst >= 0 and indexed < 0:
                        return 0
                    if indexst >= 0 and indexed >= 0:
                        return 0 if indexst < indexed else 1
                    elif indexst >= 0:
                        return 0
                    elif indexed >= 0:
                        return 1
    return -1


def test_istelework(inbox):
    members = load_members(FILE_MEMBERS)
    dt = datetime.datetime.today()
    date_from = datetime.date(dt.year, dt.month, 1)
    print(dtnew)
    for item in inbox.Items:
        r = isTelework(item, members, date_from)
        if r == 0:
            print("開始", item.senton, item.sendername)
        elif r == 1:
            print("終了", item.senton, item.sendername)
    print("test istelework done.")


# test_istelework(tele)

In [23]:
COL_DATE = 1
COL_NAME = 2
COL_START = 3
COL_END = 4
COL_DURATION = 5
COL_OVERTIME = 6
COL_PLACE = 7
COL_MEMO = 8
from openpyxl.styles import colors
from openpyxl.styles import PatternFill
from openpyxl.styles import Font, Color

# 休日判定：週末（土日）もしくは祝日の判定
def isdayoff(dt, holidays):
    if dt.weekday() > 4:
        return True
    for holiday in holidays:
        if dt == holiday:
            return True
    return False


# Excel列の幅自動設定
def adjustwidth(ws, col_index):
    for col in ws.columns:
        if col[0].column == col_index or col_index == 0:
            max_length = 0
            column = col[0].column_letter
            for cell in col:
                max_length = max(len(str(cell.value)), max_length)
            # print(column, max_length, ws.column_dimensions[column].width)
            ws.column_dimensions[column].width = (max_length + 2) * 1.2


# Excelワークシートの初期化
def init_sheet(ws, dt, members, holidays):
    ws.cell(row=1, column=COL_DATE, value="日付")
    ws.cell(row=1, column=COL_NAME, value="氏名")
    ws.cell(row=1, column=COL_START, value="開始")
    ws.cell(row=1, column=COL_END, value="終了")
    ws.cell(row=1, column=COL_DURATION, value="勤務時間")
    ws.cell(row=1, column=COL_OVERTIME, value="超勤時間")
    ws.cell(row=1, column=COL_PLACE, value="場所")
    ws.cell(row=1, column=COL_MEMO, value="備考")
    for c in range(COL_DATE, COL_MEMO + 1):
        ws.cell(row=1, column=c).fill = PatternFill("solid", fgColor="4169e1")
        ws.cell(row=1, column=c).font = Font(color=colors.WHITE)

    r = 2
    for day in range(1, calendar.monthrange(dt.year, dt.month)[1] + 1):
        d = dt.replace(day=day)
        # print(d.strftime('%m/%d'))
        for name in members:
            celldate = ws.cell(row=r, column=COL_DATE, value=d)
            celldate.number_format = "m/d(aaa)"
            cellname = ws.cell(row=r, column=COL_NAME, value=name)
            if isdayoff(d, holidays):
                for c in range(COL_DATE, COL_MEMO + 1):
                    ws.cell(row=r, column=c).fill = PatternFill(
                        "solid", fgColor="b0c4de"
                    )
                # celldate.fill = PatternFill("solid", fgColor="ffa500")
                # cellname.fill = PatternFill("solid", fgColor="ffa500")
            formulastr = '=IF(OR(ISBLANK(D{0}),ISBLANK(C{0})),"",D{0}-C{0})'.format(r)
            cellduration = ws.cell(row=r, column=COL_DURATION, value=formulastr)
            cellduration.number_format = "h:mm"
            formula_overtime = '=IFERROR(ROUND((E{0}-TIME(8,30,0))/TIME(0,15,0),0)*0.25,"")'.format(
                r
            )
            cellovertime = ws.cell(row=r, column=COL_OVERTIME, value=formula_overtime)
            cellovertime.number_format = "0.00"
            r += 1
    lastrow = r - 1
    for name in members:
        ws.cell(row=r, column=COL_DATE, value="合計")
        ws.cell(row=r, column=COL_NAME, value=name)
        formula_sum = "=SUMIF({0}$2:{0}${1},{0}{2},{3}$2:{3}${1})".format(
            openpyxl.utils.get_column_letter(COL_NAME),
            lastrow,
            r,
            openpyxl.utils.get_column_letter(COL_OVERTIME),
        )
        cellovertime = ws.cell(row=r, column=COL_OVERTIME, value=formula_sum)
        cellovertime.number_format = "0.00"
        r += 1
    filter_str = "A1:{0}{1}".format(
        openpyxl.utils.get_column_letter(COL_MEMO), len(members) * 3 + 1
    )
    ws.auto_filter.ref = filter_str
    ws.freeze_panes = "A2"


# 日付から該当のExcelワークシートを取得、なければ新規作成して初期化する
def findSheet(wb, dt, members, holidays):
    sheetname = dt.strftime("%Y_%m")
    if sheetname in wb.sheetnames:
        return wb[sheetname]
    if wb.active.title == "Sheet":
        ws = wb.active
        ws.title = sheetname
    else:
        ws = wb.create_sheet(sheetname)
    init_sheet(ws, dt.date(), members, holidays)
    adjustwidth(ws, COL_NAME)
    return ws


# ワークシート初期化のテスト
def init_test(filename):
    members = load_members(FILE_MEMBERS)
    holidays = load_holidays(FILE_HOLIDAYS)
    print(holidays)
    wb = openpyxl.Workbook()
    ws = findSheet(wb, datetime.datetime.now(), members, holidays)
    wb.save(filename)
    wb.close()
    print("int_test done")


testfilename = r"C:\Users\1027541\Documents\init_test.xlsx"
#init_test(testfilename)

In [28]:
# メールから該当行（日付と氏名）を検索、行番号を返却
def findrow(ws, message):
    for r in range(2, ws.max_row + 1):
        dt = ws.cell(row=r, column=COL_DATE).value
        if type(dt) is datetime.datetime:
            dt = dt.date()
        nm = ws.cell(row=r, column=COL_NAME).value
        if dt == message.senton.date() and nm in message.sendername:
            return r
    return 0

# メインの処理。Excelファイル読み取り/生成、メール検索、Excel記載、保存
def write_to_excel(filename, members, holidays, inbox, archive=None, create_new=False, date_from=None):
    # filename=r"C:\Users\1027541\Documents\telework.xlsx"
    if os.path.exists(filename) and not create_new:
        wb = openpyxl.load_workbook(filename)
    else:
        wb = openpyxl.Workbook()
    messages = inbox.Items
    list_move = list()
    for message in messages:
        ret = isTelework(message, members, date_from)
        if ret >= 0:
            print("\r", message.senton, message.sendername, message.subject, end="")
            ws = findSheet(wb, message.senton, members, holidays)
            r = findrow(ws, message)
            # print(ws.title,r)
            c = ws.cell(row=r, column=COL_START + ret, value=message.senton.time())
            c.number_format = "hh:mm"
            ws.cell(row=r, column=COL_PLACE, value="テレワーク")
            list_move.append(message)
            # print(str(i)+"\n"+str(i)+"\n"+str(i)+"\n"+"\033[3A",end="")
    wb.save(filename)
    print("\nsave done:", filename)
    wb.close()
    if archive != None:
        for message in list_move:
            message.UnRead = False
            message.Move(archive)


### 実行部分
* メインしたほうがいいかな

In [30]:
# FILE_OUTPUT = r"C:\Users\1027541\Documents\teleworkpy.xlsx"
# FILE_OUTPUT = r"\\10.177.108.23\bear2$\公社基推PMO\現行データ\2020_ＰＭＯ\65_グループ会社問題PJ撲滅\05_【G外秘】作業フォルダ\こぎ\teleworkpy.xlsx"
FILE_OUTPUT = r"\\10.177.108.23\bear2$\公社基推PMO\現行データ\2021_ＰＭＯ\65_グループ会社問題PJ撲滅\05_【G外秘】作業フォルダ\kogi\telework.xlsx"

# 実行部分
import sys
import argparse

members = load_members(FILE_MEMBERS)
holidays = load_holidays(FILE_HOLIDAYS)
filename = FILE_OUTPUT
tele = findTeleworkFolder(root, FOLDER_ARCHIVE)
# write_to_excel(filename,members,holidays,tele,create_new=True)
write_to_excel(filename, members, holidays, inbox, archive=tele)
#write_to_excel(filename, members, holidays, inbox)
#today = datetime.date.today()
#date_from = datetime.date(today.year,today.month,1)
#write_to_excel(filename, members, holidays, tele, create_new=True,date_from=date_from)


members: ['鈴木 一彦', '菊川 敬二', '青木 謙']
holidays: [datetime.date(2020, 1, 1), datetime.date(2020, 1, 13), datetime.date(2020, 2, 11), datetime.date(2020, 2, 23), datetime.date(2020, 2, 24)]
 2021-04-14 19:57:15+00:00 PSD 菊川 敬二/Kikukawa, Hirotsugu (NTT DATA) 【ご連絡】　テレワーク勤務終了（04/14）→ＰＭ豊洲
save done: \\10.177.108.23\bear2$\公社基推PMO\現行データ\2021_ＰＭＯ\65_グループ会社問題PJ撲滅\05_【G外秘】作業フォルダ\kogi\telework.xlsx
